### Sequential API, Functional API

#### Sequential API (단순한것)
- 간단한 모델을 구현하기에 적합하고 단순하게 층을 쌓는 방식으로 쉽고 사용하기가 간단하다.
- 단일 입력 및 출력만 있으므로 레이어를 공유하거나 여러 입력 또는 출력을 가질 수 있는 모델을 생성할 수 없다.

#### Functional API (복잡한것, 단 단순한것도 functional로도 사용함)
- Functional API는 Sequential API로는 구현하기 어려운 복잡한 모델들을 구현할 수 있다.
- 여러 개의 입력 및 출력을 가진 모델을 구현하거나 층 간의 연결 및 연산을 수행하는 모델 구현 시 사용한다.

#### 성능 평가
<img src="./images/validation.png" width="700" style="margin-left: 0">
퍼포먼스 이스티메이션(사전에 만들어진 모델을 사용할 때)  
큰 데이터 세트에서는 그냥 표준대로 해라!  
스몰 데이터 세트는 반복해서 데이터 할 수 있도록 하고 부트스트랩을 사용해서 해라! = 더 훈련할 수 있겠끔 (0.632 부트스트랩은 부투스트랩 이름)  
데이터 세트가 너무 적으니까 20%가져와서 하기도 힘듦 그래서 99개 훈련하고 1개 랜덤으로 뽑아와서 검증  


하이퍼 파라미터 오티마이제이션  
하이퍼파라미터를 최적화할 수 있따면  
데이터 개수가 많으면 세가지로 분리해서 하기  

데이터 개수가 적으면 
데이터 세트가 너무 적으니까 20%가져와서 하기도 힘듦 그래서 99개 훈련하고 1개 랜덤으로 뽑아와서 검증

시퀀셜은 ㅐㄷ괄호 열고 시작 제어문 사용 불가 \
펑셔널은 내놓고 사용 제어문 결합 가능, 상황에 따라 튜닝 가능

In [2]:
from tensorflow.keras.datasets import fashion_mnist

(train_images, train_targets), (test_images, test_targets) = fashion_mnist.load_data()

print(train_images.shape, train_targets.shape)
print(test_images.shape, test_targets.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [24]:
import numpy as np

def get_preprocessed_data(images, targets):
    images = np.array(images / 255.0, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)

    return images, targets

train_images, train_targets = get_preprocessed_data(train_images, train_targets)
test_images, test_targets = get_preprocessed_data(test_images, test_targets)

# shape는 변화가 생기면 안됨
print(train_images.shape, train_targets.shape)
print(test_images.shape, test_targets.shape)

(48000, 28, 28) (48000,)
(10000, 28, 28) (10000,)


In [8]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential

# Flatten는 층이 아니라 층에 들어가기 전에 사전 작업

INPUT_SIZE = 28

# Sequential는 list로
# 전부 Dense layer라고 부른다.

# 학습할 데이터가 많으면 neuron의 개수를 더 많이 지정(예, 64, 128, 256 등)하거나
# 은닉층 수를 더 늘리면 된다.
# 학습할 데이터가 적으면 neuron의 개수 또는 은닉층 수를 줄이는 것이 좋다.
model = Sequential([
    # 전체 28 * 28(784)개의 feature로 flatten 진행
    # 첫 번째 Input Layer
    Flatten(input_shape=(INPUT_SIZE, INPUT_SIZE)),
    
    # 두 번째 Hidden Layer
    # 2의 제곱으로 넣기(64, 128,...) = 바이어스
    Dense(64, activation='relu'),
    
    # 세 번째 Hidden Layer
    Dense(128, activation='relu'),
    
    # 마지막 Output Layer (다중)
    Dense(10, activation='softmax')
])

# 파라미터 수 = (입력 뉴런 수 * 출력 뉴런 수) + 출력 뉴런 수
# 이 파라미터 수에는 편향(bias) 파라미터도 포함되어여
model.summary()

C:\Users\LEGION\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,770 (53.79 KB)

 Trainable params: 13,770 (53.79 KB)

 Non-trainable params: 0 (0.00 B)

학습 데이터가 많으면 뉴런의 개수의 개수를 많이 층도 높이고/ > 시간이 오래걸림 .. 걍해야됨
차라리 데이터를 리셈플해서 데이터수를 잘게 쪼개거나
애초에 데이터 자체를 랜덤하게 뽑거나..
층을 건들지 ㄴㄴ

학습할 데이터가 많을 수록 복잡도가 증가
학습 데이터가 적으면 복잡도 감소 = 과적합 때문에
상황에 따라 다르지만 일단 일반적임
첫번재 사이클은 이렇게 돌리기

숫자 증가시킨 이유
그만큼 학습하라는 의미도 있지만
자극을 조금만 줘도 넘어갈 수 있게함

In [17]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy

model.compile(optimizer=Adam(0.001), loss=CategoricalCrossentropy(), metrics=['accuracy'])

In [15]:
from tensorflow.keras.utils import to_categorical

train_oh_targets = to_categorical(train_targets)
test_oh_targets = to_categorical(test_targets)

print(train_oh_targets.shape, test_oh_targets.shape)

# (원핫 인코딩)10개 피처에 60000개 행

(60000, 10) (10000, 10)


In [ ]:
# 검증 데이터와 훈련 데이터가 히스토리에 담김
# batch_size도 2의 제곱으로 가기 
history = model.fit(x=train_images, y=train_oh_targets, batch_size=64, epochs=20)

In [ ]:
history.history
# 정확도랑 loss가 나옴

In [ ]:
print(history.history['loss'])
print('=' * 80)
history.history['accuracy']

In [ ]:
# 3차원으로 변경
# 한차원 높여라
np.expand_dims(test_images[0], axis=0).shape

In [ ]:
pred_prob = model.predict(np.expand_dims(test_images[10], axis=0))
print(pred_prob)

In [ ]:
import matplotlib.pyplot as plt

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
# print(class_names[pred_prob.flatten().argsort()[::-1][0]])

plt.imshow(test_images[10], cmap='gray')
# pred_prob 가 고차원이라 플랫튼으로 피고 내림차순 정렬 0번째 방으로 예측 결과 확인
plt.title(class_names[pred_prob.flatten().argsort()[::-1][0]])
plt.show()

### Validation

In [18]:
from tensorflow.keras.datasets import fashion_mnist

(train_images, train_targets), (test_images, test_targets) = fashion_mnist.load_data()

print(train_images.shape, train_targets.shape)
print(test_images.shape, test_targets.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [19]:
import numpy as np

def get_preprocessed_data(images, targets):
    images = np.array(images / 255.0, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)

    return images, targets

train_images, train_targets = get_preprocessed_data(train_images, train_targets)
test_images, test_targets = get_preprocessed_data(test_images, test_targets)

# shape는 변화가 생기면 안됨
print(train_images.shape, train_targets.shape)
print(test_images.shape, test_targets.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [20]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

train_images, validation_images, train_targets, validation_targets = \
train_test_split(train_images, train_targets, stratify=train_targets, test_size=0.2, random_state=124)

train_oh_targets = to_categorical(train_targets)
validation_oh_targets = to_categorical(validation_targets)

print(train_images.shape, train_oh_targets.shape)
print(validation_images.shape, validation_oh_targets.shape)

(48000, 28, 28) (48000, 10)
(12000, 28, 28) (12000, 10)


In [23]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

INPUT_SIZE = 28

model = Sequential([
    Flatten(input_shape=(INPUT_SIZE, INPUT_SIZE)),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer=Adam(0.001), loss=CategoricalCrossentropy(), metrics=['acc'])

AttributeError: 'Sequential' object has no attribute 'complie'

In [ ]:
# verbose 가 올라갈 수록  간결화
# val acc가 떨어지면 과적합
history = model.fit(x=train_images, 
                    y=train_oh_targets, 
                    validation_data=(validation_images, validation_oh_targets), 
                    batch_size=32,
                    epochs=20,
                    verbose=2)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'], label='train')
plt.plot(history.history['acc'], label='train')

# 10번쯤 반복이 괜찮지 않을까...

In [ ]:
# 예측할 이미지 뽑고
# 이미지를 보고 예측된 이름 가져오기

### Functional API

In [26]:
class Test:
    def __call__(self, data):
        return data + 10

# 생성자 뒤에 괄호로 쓴다
# call 함수
print(Test()(20))

30


In [ ]:
from tensorflow.keras.layers import Layer, Input, Dense, Flatten
from tensorflow.keras.models import Model

INPUT_SIZE = 28

def create_model():
    input_tensor = Input(shape=(INPUT_SIZE, INPUT_SIZE))
    x = Flatten()(input_tensor)
    x = Dense(64, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(10, activation='softmax')(x)

    model = Model(inputs=input_tensor, output=output)
    return model

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
import numpy as np

(train_images, train_targets), (test_images, test_targets) = fashion_mnist.load_data()

def get_preprocessed_data(images, targets):
    images = np.array(images / 255.0, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)

    return images, targets


# 원핫인코디 ㅇ완료한 함수
def get_preprocessed_ohe(images, targets):
    images, targets = get_preprocessed_data(images, targets)
    oh_targets = to_categorical(targets)

    return images, oh_targets

def get_train_valid_test(train_images, train_targets, test_images, test_targets, validation_size=0.2, random)
    train_images, train_oh_targets = get_preprocessed_ohe(train_images, train_targets)
    test_images, test_oh_targets = get_preprocessed_ohe(test_images, test_targets)

    train_images, validation_images, train_oh_targets, validation_oh_targets = \
    train_test_split(train_images, train_oh_targets, stratify=train_oh_targets, test_sze=validation_images)

    return (train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_targets)


# 딥러닝은 무조건 모듈화

In [ ]:
from tensorflow.keras.datasets import fashion_mnist

(train_images, train_targets), (test_images, test_targets) = fashion_mnist.load_data()

# 케라스에서 지원하는 데이터에서만 사용가능한 모듈(케글 이런곳에서 들고온 거 안됨)
(train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_targets)
get_train_valid_test(train_images, train_targets, test_images, test_targets)

print(train_images.shape, train_oh_targets.shape)
print(validation_images.shape, validation_oh_targets.shape)
print(test_images.shape, test_oh_targets.shape)

In [ ]:
model = create_model()

model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

# 아래꺼는 쓰고 마는 것... 어디 담지 말기
model.compile(optimizer=Adam(), loss=

In [ ]:
model.fit(x=train_images, y=train_oh_targets, batch_size=64, epochs=20, validation_data=(validation_images, validation_oh_targets))

In [ ]:
import matplotlib.pyplot import plt

def show_history(history):
    plt.plt